#### Retrieve environment variables

In [1]:
import os
from dotenv import find_dotenv, load_dotenv

env_file = find_dotenv(".env")
load_dotenv(env_file)

DBHOST = os.environ.get("DBHOST")
DBPORT = os.environ.get("DBPORT")
DBUSER = os.environ.get("DBUSER")
DBNAME = os.environ.get("DBNAME")
DBPASS = os.environ.get("DBPASS")

#### Create tables

In [2]:
from sqlalchemy import URL, create_engine
from sqlalchemy.orm import sessionmaker
from sql.schema import Base, Countries, Locations, Parameters, Sensors
from sqlalchemy.dialects.postgresql import insert

In [3]:
engine_url = URL.create(
    drivername="postgresql+psycopg2",
    username=DBUSER,
    password=DBPASS,
    host=DBHOST,
    port=DBPORT,
    database=DBNAME,
)
engine = create_engine(url=engine_url)

Base.metadata.create_all(bind=engine)
Session = sessionmaker(bind=engine)
session = Session()


#### Set up openAQ api app

In [4]:
import sys
import os

# Get the current working directory (the directory where the notebook is running)
notebook_dir = os.getcwd()

# Construct the path to the 'src' folder
src_dir = os.path.join(notebook_dir, 'src')

# Add 'src' folder to the system path
sys.path.append(src_dir)

from src.openAQ_api import openAQ

app = openAQ()

#### Populate Parameters Table

In [5]:
params = app.get_parameters()
params

{'meta': {'name': 'openaq-api',
  'website': '/',
  'page': 1,
  'limit': 100,
  'found': 38},
 'results': [{'id': 1,
   'name': 'pm10',
   'units': 'µg/m³',
   'displayName': 'PM10',
   'description': 'Particulate matter less than 10 micrometers in diameter mass concentration'},
  {'id': 2,
   'name': 'pm25',
   'units': 'µg/m³',
   'displayName': 'PM2.5',
   'description': 'Particulate matter less than 2.5 micrometers in diameter mass concentration'},
  {'id': 3,
   'name': 'o3',
   'units': 'µg/m³',
   'displayName': 'O₃ mass',
   'description': 'Ozone mass concentration'},
  {'id': 4,
   'name': 'co',
   'units': 'µg/m³',
   'displayName': 'CO mass',
   'description': 'Carbon Monoxide mass concentration'},
  {'id': 5,
   'name': 'no2',
   'units': 'µg/m³',
   'displayName': 'NO₂ mass',
   'description': 'Nitrogen Dioxide mass concentration'},
  {'id': 6,
   'name': 'so2',
   'units': 'µg/m³',
   'displayName': 'SO₂ mass',
   'description': 'Sulfur Dioxide mass concentration'},
  {'

In [6]:
param_list = []

for param in params["results"]:
    param_list.append({
            "parameter_id": param["id"],
            "parameter_name":param["name"],
            "units":param["units"],
            "display_name":param["displayName"],
            "description":param["description"],
        }
    )

In [7]:
session.execute(insert(Parameters).values(param_list).on_conflict_do_nothing())
session.commit()

#### Populate Countries Table

OpenAQ uses the [ISO 3166-1 alpha 2](https://www.iso.org/obp/ui/#search) country codes, which may differ from that reported by the upstream provider. Place the countries of interest inside the `relevant_countries` tuple.

In [8]:
relevant_countries = ("HK", "SG", "TW", "VN", "TH", "PH", "ID", "MY")

In [9]:
countries = app.get_countries(limit=200)
countries

{'meta': {'name': 'openaq-api',
  'website': '/',
  'page': 1,
  'limit': 200,
  'found': 131},
 'results': [{'id': 1,
   'code': 'ID',
   'name': 'Indonesia',
   'datetimeFirst': '2016-01-30T01:00:00Z',
   'datetimeLast': '2024-10-21T07:00:00Z',
   'parameters': [{'id': 1,
     'name': 'pm10',
     'units': 'µg/m³',
     'displayName': None},
    {'id': 2, 'name': 'pm25', 'units': 'µg/m³', 'displayName': None},
    {'id': 3, 'name': 'o3', 'units': 'µg/m³', 'displayName': None},
    {'id': 10, 'name': 'o3', 'units': 'ppm', 'displayName': None},
    {'id': 11, 'name': 'bc', 'units': 'µg/m³', 'displayName': None},
    {'id': 15, 'name': 'no2', 'units': 'ppb', 'displayName': None},
    {'id': 19, 'name': 'pm1', 'units': 'µg/m³', 'displayName': None},
    {'id': 21, 'name': 'co2', 'units': 'ppm', 'displayName': None},
    {'id': 24, 'name': 'no', 'units': 'ppb', 'displayName': None},
    {'id': 98, 'name': 'relativehumidity', 'units': '%', 'displayName': None},
    {'id': 100, 'name': 'tem

In [10]:
country_list = []

for country in countries["results"]:
    if country["code"] in relevant_countries:
        country_list.append({
            "country_id": country["id"],
            "country_name": country["name"],
            "first_date": country["datetimeFirst"],
            "last_date": country["datetimeFirst"],
        })


In [11]:
session.execute(insert(Countries).values(country_list).on_conflict_do_nothing())
session.commit()

#### Populate Locations Table

In [12]:
country_ids = [country["country_id"] for country in country_list]
country_ids

[1, 2, 56, 111, 167, 183, 189, 231]

In [13]:
def handle_date(datetime):
    if datetime:
        return datetime["utc"]
    else:
        return None

def get_locations(country_ids: list, app: openAQ):
    page = 1

    while True:
        locations = app.get_locations(countries_id=country_ids, limit=1000, page=page)
        print(len(locations["results"]))
        try:
            for location in locations["results"]:
                yield {
                    "location_id": location["id"],
                    "location_name": location["name"],
                    "latitude": location["coordinates"]["latitude"],
                    "longitude": location["coordinates"]["longitude"],
                    "first_date": handle_date(location["datetimeFirst"]),
                    "last_date": handle_date(location["datetimeLast"]),
                    "country_id": location["country"]["id"],
                }
            if locations["meta"]["limit"] <= locations["meta"]["found"]:
                page += 1
            else: 
                break
        except TypeError:
            page += 1
            continue


In [14]:
locations_list = []
locations_list = [detail for detail in get_locations(country_ids=country_ids, app=app)]
len(locations_list)

719


719

In [15]:
session.execute(insert(Locations).values(locations_list).on_conflict_do_nothing())
session.commit()

#### Populate Sensors Table

In [16]:
country_ids

[1, 2, 56, 111, 167, 183, 189, 231]

In [17]:
# Just a repeat of an earlier function but with different output
def get_sensors(country_ids: list, app: openAQ):
    page = 1

    while True:
        locations = app.get_locations(countries_id=country_ids, limit=1000, page=page)
        try:
            for location in locations["results"]:
                for sensor in location["sensors"]:
                    yield {
                        "sensor_id": sensor["id"],
                        "parameter_id": sensor["parameter"]["id"]
                    }
            if locations["meta"]["limit"] <= locations["meta"]["found"]:
                page += 1
            else: 
                break
        except TypeError:
            page += 1
            continue


In [18]:
sensor_list = [sensor_detail for sensor_detail in get_sensors(country_ids=country_ids, app=app)]
len(sensor_list)

3284

In [19]:
session.execute(insert(Sensors).values(sensor_list).on_conflict_do_nothing())
session.commit()